In [2]:
import pandas as pd
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split

In [3]:
DIR_INPUT = '/kaggle/input/global-wheat-detection'
DIR_INPUT = "../data"
DIR_TRAIN = f'{DIR_INPUT}/train'
DIR_TEST = f'{DIR_INPUT}/test'

locations = ['arvalis_1', 'arvalis_2', 'arvalis_3', 'inrae_1', 'usask_1', 'rres_1', 'ethz_1']

In [4]:
train_df = pd.read_csv(f"{DIR_INPUT}/train.csv")
train_df = pd.read_csv(f"{DIR_INPUT}/fixed_train.csv")

locations = train_df['source'].unique()
locations

array(['usask_1', 'arvalis_1', 'inrae_1', 'ethz_1', 'arvalis_3', 'rres_1',
       'arvalis_2'], dtype=object)

In [5]:
train_df.head()

,image_id,width,height,bbox,source
0,b6ab77fd7,1024,1024,"[834.0, 222.0, 56.0, 36.0]",usask_1
1,b6ab77fd7,1024,1024,"[226.0, 548.0, 130.0, 58.0]",usask_1
2,b6ab77fd7,1024,1024,"[377.0, 504.0, 74.0, 160.0]",usask_1
3,b6ab77fd7,1024,1024,"[834.0, 95.0, 109.0, 107.0]",usask_1
4,b6ab77fd7,1024,1024,"[26.0, 144.0, 124.0, 117.0]",usask_1


In [6]:
image_ids = train_df['image_id'].unique()
len(image_ids)

3373

In [7]:
train_df.groupby(train_df['source'])['image_id'].value_counts()

source     image_id 
arvalis_1  134d4a01c    101
           4cf91995d     97
           a06241eb3     95
           a119a31fb     95
           ab026459d     93
                       ... 
usask_1    a76ca8630      9
           bff5440bc      9
           8f05b9e06      8
           be5ba0745      8
           c3c03e0b7      8
Name: image_id, Length: 3373, dtype: int64

In [8]:
for num in range(len(locations)):
    train_df_ = train_df[train_df['source'] == locations[num]]
    image_ids = train_df_['image_id'].unique()    
    print(f'location: {locations[num]}, unique images: {len(image_ids)}')

location: usask_1, unique images: 200
location: arvalis_1, unique images: 1055
location: inrae_1, unique images: 176
location: ethz_1, unique images: 747
location: arvalis_3, unique images: 559
location: rres_1, unique images: 432
location: arvalis_2, unique images: 204


In [9]:
3373*0.2

674.6

In [10]:
# fold 0 arvalis_3 + arvalis_1 part
train_df['fold'] = -1
train_df['fold'][train_df['source'] == 'arvalis_3'] = 0

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
train_df_ = train_df[train_df['source'] == 'arvalis_1']
image_ids = train_df_['image_id'].unique()   

_, X_test = train_test_split(image_ids, test_size=(675-559), random_state=42)
len(X_test)

116

In [12]:
train_df.loc[train_df['image_id'].isin(X_test), 'fold'] = 0
train_df.loc[train_df['image_id'].isin(X_test), 'fold'].head()

6231    0
6232    0
6233    0
6234    0
6235    0
Name: fold, dtype: int64

In [13]:
train_df[train_df['source'] == 'arvalis_3'].head()

,image_id,width,height,bbox,source,fold
106656,aadb83e1d,1024,1024,"[919.0, 33.0, 105.0, 105.0]",arvalis_3,0
106657,aadb83e1d,1024,1024,"[568.0, 17.0, 95.0, 187.0]",arvalis_3,0
106658,aadb83e1d,1024,1024,"[739.0, 501.0, 154.0, 85.0]",arvalis_3,0
106659,aadb83e1d,1024,1024,"[286.0, 949.0, 100.0, 75.0]",arvalis_3,0
106660,aadb83e1d,1024,1024,"[657.0, 660.0, 141.0, 172.0]",arvalis_3,0


In [14]:
imgs = train_df['image_id'][train_df['fold'] == 0].unique()
len(imgs)

675

In [15]:
# fold 1 rres_1, arvalis_2 and arvalis_1 part
train_df['fold'][train_df['source'] == 'rres_1'] = 1
train_df['fold'][train_df['source'] == 'arvalis_2'] = 1

_, X_test = train_test_split(image_ids, test_size=(675-432-204), random_state=42)
len(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


39

In [16]:
train_df.loc[train_df['image_id'].isin(X_test), 'fold'] = 1
imgs = train_df['image_id'][train_df['fold'] == 1].unique()
len(imgs)

675

In [17]:
# fold 2 ethz_1 almoust all
train_df_ = train_df[train_df['source'] == 'ethz_1']
image_ids = train_df_['image_id'].unique()  

_, X_test = train_test_split(image_ids, test_size=(675), random_state=42)
len(X_test)

675

In [18]:
train_df.loc[train_df['image_id'].isin(X_test), 'fold'] = 2
imgs = train_df['image_id'][train_df['fold'] == 2].unique()
len(imgs)

675

In [19]:
# fold 3 usask_1, inrae_1 and arvalis_1 part
train_df['fold'][train_df['source'] == 'usask_1'] = 3
train_df['fold'][train_df['source'] == 'inrae_1'] = 3

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [21]:
train_df_ = train_df[(train_df['source'] == 'arvalis_1')&(train_df['fold'] == -1)]
image_ids = train_df_['image_id'].unique()
len(image_ids)

939

In [22]:
_, X_test = train_test_split(image_ids, test_size=(675-376), random_state=42)
len(X_test)
train_df.loc[train_df['image_id'].isin(X_test), 'fold'] = 3
imgs = train_df['image_id'][train_df['fold'] == 3].unique()
len(imgs)

675

In [23]:
train_df.to_csv('tanya_folds.csv')

In [ ]:
def make_folds(train_df: pd.DataFrame) -> pd.DataFrame:
    """
    Alex folds
    """
    bboxs = np.stack(train_df['bbox'].apply(lambda x: np.fromstring(x[1:-1], sep=',')))
    for i, column in enumerate(['x', 'y', 'w', 'h']):
        marking[column] = bboxs[:,i]
    marking.drop(columns=['bbox'], inplace=True)
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    df_folds = marking[['image_id']].copy()
    df_folds.loc[:, 'bbox_count'] = 1
    df_folds = df_folds.groupby('image_id').count()
    df_folds.loc[:, 'source'] = marking[['image_id', 'source']].groupby('image_id').min()['source']
    df_folds.loc[:, 'stratify_group'] = np.char.add(
        df_folds['source'].values.astype(str),
        df_folds['bbox_count'].apply(lambda x: f'_{x // 15}').values.astype(str)
    )
    df_folds.loc[:, 'fold'] = 0

    for fold_number, (train_index, val_index) in enumerate(skf.split(X=df_folds.index, y=df_folds['stratify_group'])):
        df_folds.loc[df_folds.iloc[val_index].index, 'fold'] = fold_number

    return df_folds    
